In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [8]:
import pandas as pd
import numpy as np

KNN

# Read the CSV and Perform Basic Data Cleaning

In [9]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [10]:
df['koi_disposition'].unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [17]:
print(df['koi_period'].nunique())

6991


In [18]:
df.shape

(6991, 41)

# Select your features (columns)

In [19]:
# Set features. This will also be used as your x values.
Actual_Results = df["koi_disposition"]
Feature_Set = df.drop("koi_disposition", axis=1)
#Feature_Set
Feature_Names = Feature_Set.columns
Feature_Names

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk',
       'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1',
       'koi_impact_err2', 'koi_duration', 'koi_duration_err1',
       'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
       'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg',
       'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1',
       'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

In [20]:
#using Actual_Results as the y-values
#Actual_Results
X=Feature_Set
y=Actual_Results

In [21]:
#identifying features 
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.13739363, 0.16851492, 0.12570931, 0.05697643, 0.01749121,
       0.01501246, 0.01472689, 0.01300488, 0.02036338, 0.02093628,
       0.01151841, 0.01167328, 0.01029926, 0.01659331, 0.02769283,
       0.03096716, 0.01829239, 0.01174324, 0.01229125, 0.01459339,
       0.0108451 , 0.00991869, 0.01315288, 0.00914777, 0.00925674,
       0.00853783, 0.03192964, 0.01010253, 0.01000547, 0.01874775,
       0.01614379, 0.0111962 , 0.01045639, 0.01440561, 0.00890809,
       0.01062069, 0.00929914, 0.01126354, 0.01002058, 0.01024767])

In [22]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.168515
koi_fpflag_nt        0.137394
koi_fpflag_co        0.125709
koi_fpflag_ec        0.056976
koi_model_snr        0.031930
koi_duration_err2    0.030967
koi_duration_err1    0.027693
koi_time0bk_err2     0.020936
koi_time0bk_err1     0.020363
koi_steff_err1       0.018748
dtype: float64

In [23]:
# Set features based on feature importances
X = df[feat_imp.index]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [26]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)